# Ex01 고속도로

In [10]:
import pandas as pd  
df_csv1 = pd.read_csv('ds_highway_01_07.csv', encoding='utf-8', sep=',')
df_csv2 = pd.read_csv('ds_highway_08_12.csv', encoding='utf-8', sep=',')

In [12]:
df_csv2['StartPoint'] = df_csv2['StartPoint'].str.upper()
df_csv2

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140801,GANGWON,63365.0,7044.0,892.0,1756.0,140258.0
1,20140801,GYEONGGI,553781.0,110895.0,9961.0,12596.0,89665.0
2,20140801,GYEONGSANG,11911.0,7462.0,21988.0,429499.0,2447.0
3,20140801,JEOLLA,5764.0,6526.0,171004.0,18912.0,1203.0
4,20140801,CHUNGCHEONG,104192.0,165190.0,7846.0,7936.0,9991.0
...,...,...,...,...,...,...,...
760,20141231,GANGWON,45095.0,5978.0,676.0,1243.0,81547.0
761,20141231,GYEONGGI,558204.0,89242.0,4809.0,9903.0,59000.0
762,20141231,GYEONGSANG,9604.0,5657.0,14685.0,380287.0,1538.0
763,20141231,JEOLLA,3679.0,4224.0,134007.0,14128.0,640.0


In [13]:
df = pd.concat([df_csv1, df_csv2])
df

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,CHUNGCHEONG,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,GANGWON,57398.0,5783.0,427.0,991.0,102850.0
2,20140101,GYEONGGI,373199.0,82263.0,3114.0,5841.0,51386.0
3,20140101,GYEONGSANG,6926.0,5368.0,15514.0,334217.0,986.0
4,20140101,JEOLLA,3298.0,4759.0,130474.0,14832.0,478.0
...,...,...,...,...,...,...,...
760,20141231,GANGWON,45095.0,5978.0,676.0,1243.0,81547.0
761,20141231,GYEONGGI,558204.0,89242.0,4809.0,9903.0,59000.0
762,20141231,GYEONGSANG,9604.0,5657.0,14685.0,380287.0,1538.0
763,20141231,JEOLLA,3679.0,4224.0,134007.0,14128.0,640.0


# no.1 : quantile

In [ ]:
df1 = df[df['StartPoint'] == 'GANGWON']
df1 = pd.concat([df1['Gyeonggi'], df1['Chungcheong'], df1['Jeolla'], df1['Gyeongsang'], df1['Gangwon']])
df1
ans1 = df1.quantile(q=[0.25, 0.5, 0.75])
print(f'[1] {int(round(ans1[0.25]))}, {int(round(ans1[0.50]))}, {int(round(ans1[0.75]))}') 
# f' 는 f-string표현방식, https://www.python.org/dev/peps/pep-0498/

In [ ]:
import datetime
date = datetime.datetime.now()

# f'{date:%Y-%m-%d} is on a {date:%A}'


# no.2 : t test

In [ ]:
df_gi = df[df['StartPoint'] == 'GYEONGGI']['Gyeongsang']
df_sang = df[df['StartPoint'] == 'GYEONGSANG']['Gyeonggi']
from scipy import stats
ans2_1 = stats.ttest_ind(df_gi, df_sang)
ans2_2 = "YES" if ans2_1.pvalue <= 0.05 else "NO"
print(f'{round(ans2_1.pvalue, 4)}, {ans2_2}' )

# no.3 :  K-means clustering

In [ ]:
df3 = df[df['StartPoint']=='CHUNGCHEONG'][['date','Gangwon']]
df3
import datetime
df3['weekday'] = df3['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').weekday())
df3['month'] = df3['date'].apply(lambda x: str(x) [4:6])
df3_group = df3.groupby(['weekday', 'month']).agg({'Gangwon': 'mean'})
df3_group.reset_index(inplace=True)

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
arr3 = pd.DataFrame()

for i in range(7):
    temp = mms.fit_transform(df3_group[df3_group['weekday']==i][['Gangwon']])
    arr3 = pd.concat([arr3, pd.DataFrame(temp)])
arr3.reset_index(inplace=True, drop=True)
arr3.rename(columns={arr3.columns[0]: 'mms'}, inplace=True)
df3_1 = pd.concat([df3_group, arr3], axis=1)
df3_2 = df3_1.groupby(['month']).agg({'mms' : 'mean'})
df3_2.reset_index(inplace=True)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, max_iter=100, random_state=1234).fit(df3_2[['mms']])

df3_3 = pd.concat([df3_2, pd.DataFrame(kmeans.predict(df3_2[['mms']]), columns=['cls'])], axis=1)
ans3_1 = len(df3_3[df3_3['cls'] == 1])

m  = df3_3[df3_3['cls'] == 1]['month']
temp = df3_1[df3_1['month'].isin(list(m))]
ans3_2 = temp[temp['weekday'] == 0]['mms'].mean()
print( f' {ans3_1}, {round(ans3_2, 2)}')

#df3_group